<a href="https://colab.research.google.com/github/AbsorbentKarma/Clase/blob/main/M2_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Greedy 1

Esta parte del codigo solo es para poder interpretar el problema en forma de codigo, la forma en la que se pidio esto en chatGPT fue tomando una foto de la tabla con los datos necesarios y dando algunos de los datos del problema para asegurar que se entendiera el uso de nodos durante este trabajo.

In [ ]:
import pandas as pd
import itertools

# Crear un diccionario con los datos de la tabla
data = {
    'Conexión': [f'e{i+1}' for i in range(45)],
    'Ciudades Conectadas': [
        'A-B', 'A-C', 'A-D', 'A-E', 'A-F', 'A-G', 'A-H', 'A-I', 'A-J',
        'B-C', 'B-D', 'B-E', 'B-F', 'B-G', 'B-H', 'B-I', 'B-J',
        'C-D', 'C-E', 'C-F', 'C-G', 'C-H', 'C-I', 'C-J',
        'D-E', 'D-F', 'D-G', 'D-H', 'D-I', 'D-J',
        'E-F', 'E-G', 'E-H', 'E-I', 'E-J',
        'F-G', 'F-H', 'F-I', 'F-J',
        'G-H', 'G-I', 'G-J',
        'H-I', 'H-J',
        'I-J'
    ],
    'Ancho de Banda (Mbps)': [
        200, 250, 220, 280, 160, 240, 230, 220, 210,
        170, 210, 190, 160, 190, 240, 220, 220,
        180, 200, 190, 170, 200, 180, 220,
        220, 180, 200, 190, 210, 230,
        230, 210, 270, 280, 270,
        220, 200, 190, 280,
        210, 290, 180,
        200, 290, 210
    ]
}

# Crear el dataframe
df = pd.DataFrame(data)

# Mostrar el dataframe
print(df)

   Conexión Ciudades Conectadas  Ancho de Banda (Mbps)
0        e1                 A-B                    200
1        e2                 A-C                    250
2        e3                 A-D                    220
3        e4                 A-E                    280
4        e5                 A-F                    160
5        e6                 A-G                    240
6        e7                 A-H                    230
7        e8                 A-I                    220
8        e9                 A-J                    210
9       e10                 B-C                    170
10      e11                 B-D                    210
11      e12                 B-E                    190
12      e13                 B-F                    160
13      e14                 B-G                    190
14      e15                 B-H                    240
15      e16                 B-I                    220
16      e17                 B-J                    220
17      e1

Para la primera solucion greedy se uso el modelo de respuesta ideal la cual se conforma de los siguientes pasos

- Ordenar los nodos segun su peso en mbps
- Iniciar un conjunto vacio en el cual se guardan los resultados que se vayan acumulando
- Ordenar los nodos segun el orden dado anteriormente asegurando que no se tengan ciclos en ningun momento
- Repetir el proceso hasta haber cubierto todos los nodos

In [ ]:
# Ordenar las aristas en orden decreciente de ancho de banda
df_sorted = df.sort_values(by='Ancho de Banda (Mbps)', ascending=False).reset_index(drop=True)

# Inicializar el conjunto T para las aristas del árbol de expansión
T = []

# Función para encontrar el conjunto de un nodo (uso de Union-Find)
def find(parent, i):
    if parent[i] == i:
        return i
    else:
        return find(parent, parent[i])

# Función para unir dos subconjuntos
def union(parent, rank, x, y):
    xroot = find(parent, x)
    yroot = find(parent, y)

    if rank[xroot] < rank[yroot]:
        parent[xroot] = yroot
    elif rank[xroot] > rank[yroot]:
        parent[yroot] = xroot
    else:
        parent[yroot] = xroot
        rank[xroot] += 1

# Inicializar el conjunto padre y el rango
num_vertices = len(set(df['Ciudades Conectadas'].str.split('-').sum()))
parent = [i for i in range(num_vertices)]
rank = [0] * num_vertices

# Mapeo de ciudades a índices
city_to_index = {city: index for index, city in enumerate(set(df['Ciudades Conectadas'].str.split('-').sum()))}

# Iterar sobre las aristas ordenadas
for _, row in df_sorted.iterrows():
    u, v = row['Ciudades Conectadas'].split('-')
    u_index = city_to_index[u]
    v_index = city_to_index[v]

    # Verificar si se forma un ciclo
    if find(parent, u_index) != find(parent, v_index):
        T.append(row)
        union(parent, rank, u_index, v_index)

    # Detenerse si T contiene |V| - 1 aristas
    if len(T) == num_vertices - 1:
        break

# Mostrar las conexiones seleccionadas y el ancho de banda total
print("Conexiones seleccionadas para el árbol de expansión máximo:")
for edge in T:
    print(f"{edge['Ciudades Conectadas']} : {edge['Ancho de Banda (Mbps)']} Mbps")


# Sumar el ancho de banda total de las conexiones seleccionadas
total_bandwidth = sum(edge['Ancho de Banda (Mbps)'] for edge in T)
print(f"\nAncho de banda total de la solución actual: {total_bandwidth} Mbps")

Conexiones seleccionadas para el árbol de expansión máximo:
G-I : 290 Mbps
H-J : 290 Mbps
A-E : 280 Mbps
E-I : 280 Mbps
F-J : 280 Mbps
E-H : 270 Mbps
A-C : 250 Mbps
B-H : 240 Mbps
D-J : 230 Mbps

Ancho de banda total de la solución actual: 2410 Mbps


Para la busqueda local de este codigo se busco hacer una eliminacion de 2 de los nodos usados en nuestra solucion ideal a la vez, y con estas eliminaciones se probaron el resto de las opciones disponibles, en caso de observarse una mejor opcion se opta por esta.

Prompt: ahora con esto quiero intentar encontrar una mejor solucion, asi que elimina de 2 en 2 las conexiones que se tienen actualmente y intenta todas las opciones posibles que se tengan. Recuerda que en la respuesta final solo pueden llegar a cambiar 2 partes de la solucion que ya se tiene

In [ ]:
total_bandwidth = sum(edge['Ancho de Banda (Mbps)'] for edge in T)
print(f"\nAncho de banda total: {total_bandwidth} Mbps")

# Función para verificar si un conjunto de aristas forma un árbol de expansión
def is_valid_tree(edges, num_vertices):
    parent = [i for i in range(num_vertices)]
    rank = [0] * num_vertices

    for edge in edges:
        u, v = edge['Ciudades Conectadas'].split('-')
        u_index = city_to_index[u]
        v_index = city_to_index[v]

        if find(parent, u_index) == find(parent, v_index):
            return False  # Crea un ciclo
        union(parent, rank, u_index, v_index)

    return len(edges) == num_vertices - 1  # Debe haber |V| - 1 aristas

# Guardar la solución actual
current_solution = T.copy()
current_bandwidth = total_bandwidth

# Inicializar variables para almacenar la mejor solución encontrada
best_solution = current_solution.copy()
best_bandwidth = current_bandwidth

# Probar eliminar combinaciones de aristas y agregar nuevas
for pair in itertools.combinations(current_solution, 2):
    # Crear una nueva solución eliminando las aristas del par
    new_solution = [edge for edge in current_solution if edge['Ciudades Conectadas'] not in [pair[0]['Ciudades Conectadas'], pair[1]['Ciudades Conectadas']]]

    # Intentar añadir nuevas conexiones que no estén en la solución actual
    for _, row in df_sorted.iterrows():
        # Solo agregar filas que no estén en new_solution
        if row['Ciudades Conectadas'] not in [edge['Ciudades Conectadas'] for edge in new_solution]:
            new_solution.append(row)  # Agregar la nueva conexión

            # Verificar si la nueva solución es un árbol de expansión válido
            if is_valid_tree(new_solution, num_vertices):
                new_bandwidth = sum(edge['Ancho de Banda (Mbps)'] for edge in new_solution)

                # Comparar con la mejor solución encontrada
                if new_bandwidth > best_bandwidth:
                    best_bandwidth = new_bandwidth
                    best_solution = new_solution.copy()

            new_solution.pop()  # Volver a eliminar la conexión agregada de la última posición

# Imprimir la mejor solución encontrada
print("Mejor solución encontrada:")
for edge in best_solution:
    print(f"{edge['Ciudades Conectadas']} : {edge['Ancho de Banda (Mbps)']} Mbps")

print(f"\nAncho de banda total de la mejor solución: {best_bandwidth} Mbps")


Ancho de banda total: 2410 Mbps
Mejor solución encontrada:
G-I : 290 Mbps
H-J : 290 Mbps
A-E : 280 Mbps
E-I : 280 Mbps
F-J : 280 Mbps
E-H : 270 Mbps
A-C : 250 Mbps
B-H : 240 Mbps
D-J : 230 Mbps

Ancho de banda total de la mejor solución: 2410 Mbps


## Exhaustivo
Este metodo se uso para verificar si la respuesta greedy conseguida anteriormente era la respuesta mas optima, siendo que al intentar realizar la busqueda local no se tuvo ninguna clase de mejora.

Prompt: Con este codigo quiero que hagas una examinacion de todas las posibilidades de nodos que se puedan hacer. Como limitante se tiene que saber que este problema esta inspirado por el Problema del  ́Arbol de Expansi ́on de M ́axima  Suma de Pesos asi que la solucion tiene que estar puesta con los limitantes ya decididos en ese tipo de problema los cuales son:

-Todos los nodos se conectan
- No se tienen ciclos
- se consigue el valor maximo de mbps

Copy paste del codigo usado anteriormente

In [ ]:
import pandas as pd
import itertools
import networkx as nx

# Crear un diccionario con los datos de la tabla
data = {
    'Conexión': [f'e{i+1}' for i in range(45)],
    'Ciudades Conectadas': [
        'A-B', 'A-C', 'A-D', 'A-E', 'A-F', 'A-G', 'A-H', 'A-I', 'A-J',
        'B-C', 'B-D', 'B-E', 'B-F', 'B-G', 'B-H', 'B-I', 'B-J',
        'C-D', 'C-E', 'C-F', 'C-G', 'C-H', 'C-I', 'C-J',
        'D-E', 'D-F', 'D-G', 'D-H', 'D-I', 'D-J',
        'E-F', 'E-G', 'E-H', 'E-I', 'E-J',
        'F-G', 'F-H', 'F-I', 'F-J',
        'G-H', 'G-I', 'G-J',
        'H-I', 'H-J',
        'I-J'
    ],
    'Ancho de Banda (Mbps)': [
        200, 250, 220, 280, 160, 240, 230, 220, 210,
        170, 210, 190, 160, 190, 240, 220, 220,
        180, 200, 190, 170, 200, 180, 220,
        220, 180, 200, 190, 210, 230,
        230, 210, 270, 280, 270,
        220, 200, 190, 280,
        210, 290, 180,
        200, 290, 210
    ]
}

# Crear el dataframe
df = pd.DataFrame(data)

# Mostrar el dataframe
print(df)

   Conexión Ciudades Conectadas  Ancho de Banda (Mbps)
0        e1                 A-B                    200
1        e2                 A-C                    250
2        e3                 A-D                    220
3        e4                 A-E                    280
4        e5                 A-F                    160
5        e6                 A-G                    240
6        e7                 A-H                    230
7        e8                 A-I                    220
8        e9                 A-J                    210
9       e10                 B-C                    170
10      e11                 B-D                    210
11      e12                 B-E                    190
12      e13                 B-F                    160
13      e14                 B-G                    190
14      e15                 B-H                    240
15      e16                 B-I                    220
16      e17                 B-J                    220
17      e1

In [ ]:
# Crear un grafo vacío
G = nx.Graph()

# Añadir nodos y aristas al grafo
for index, row in df.iterrows():
    ciudad1, ciudad2 = row['Ciudades Conectadas'].split('-')
    G.add_edge(ciudad1, ciudad2, weight=row['Ancho de Banda (Mbps)'])

# Aplicar el algoritmo de Kruskal para encontrar el árbol de expansión máximo
max_span_tree = nx.maximum_spanning_tree(G)

# Calcular el valor total de Mbps
total_mbps = sum(edge[2]['weight'] for edge in max_span_tree.edges(data=True))

# Mostrar las conexiones seleccionadas en el árbol de expansión máximo
print("Conexiones en el Árbol de Expansión Máximo:")
for edge in max_span_tree.edges(data=True):
    print(f"{edge[0]}-{edge[1]}: {edge[2]['weight']} Mbps")

print(f"\nValor total de Mbps: {total_mbps} Mbps")

Conexiones en el Árbol de Expansión Máximo:
A-E: 280 Mbps
A-C: 250 Mbps
B-H: 240 Mbps
D-J: 230 Mbps
E-I: 280 Mbps
E-H: 270 Mbps
F-J: 280 Mbps
G-I: 290 Mbps
H-J: 290 Mbps

Valor total de Mbps: 2410 Mbps


## Greedy 2
Se usa el mismo codigo inicial para leer los datos dados.

In [1]:
import pandas as pd
import itertools

# Crear un diccionario con los datos de la tabla
data = {
    'Conexión': [f'e{i+1}' for i in range(45)],
    'Ciudades Conectadas': [
        'A-B', 'A-C', 'A-D', 'A-E', 'A-F', 'A-G', 'A-H', 'A-I', 'A-J',
        'B-C', 'B-D', 'B-E', 'B-F', 'B-G', 'B-H', 'B-I', 'B-J',
        'C-D', 'C-E', 'C-F', 'C-G', 'C-H', 'C-I', 'C-J',
        'D-E', 'D-F', 'D-G', 'D-H', 'D-I', 'D-J',
        'E-F', 'E-G', 'E-H', 'E-I', 'E-J',
        'F-G', 'F-H', 'F-I', 'F-J',
        'G-H', 'G-I', 'G-J',
        'H-I', 'H-J',
        'I-J'
    ],
    'Ancho de Banda (Mbps)': [
        200, 250, 220, 280, 160, 240, 230, 220, 210,
        170, 210, 190, 160, 190, 240, 220, 220,
        180, 200, 190, 170, 200, 180, 220,
        220, 180, 200, 190, 210, 230,
        230, 210, 270, 280, 270,
        220, 200, 190, 280,
        210, 290, 180,
        200, 290, 210
    ]
}

# Crear el dataframe
df = pd.DataFrame(data)

# Mostrar el dataframe
print(df)

   Conexión Ciudades Conectadas  Ancho de Banda (Mbps)
0        e1                 A-B                    200
1        e2                 A-C                    250
2        e3                 A-D                    220
3        e4                 A-E                    280
4        e5                 A-F                    160
5        e6                 A-G                    240
6        e7                 A-H                    230
7        e8                 A-I                    220
8        e9                 A-J                    210
9       e10                 B-C                    170
10      e11                 B-D                    210
11      e12                 B-E                    190
12      e13                 B-F                    160
13      e14                 B-G                    190
14      e15                 B-H                    240
15      e16                 B-I                    220
16      e17                 B-J                    220
17      e1

Ahora con este codigo para nuestra segunda solucion se usara un acercamiento diferente.

El metodo que se usara sera el usar inicialmente el nodo con un mayor valor de mbps, y seguido de esto el arbol de expansion se seguira usando los nodos que se esten usando actualmente, buscando constantemente el siguiente nodo con el valor mas alto entre las opciones disponibles hasta que se cubran todos los nodos necesarios para generar una respuesta sin tener un ciclo.

In [3]:
import pandas as pd
import networkx as nx

# Crear el dataframe con los datos ya definidos
data = {
    'Conexión': [f'e{i+1}' for i in range(45)],
    'Ciudades Conectadas': [
        'A-B', 'A-C', 'A-D', 'A-E', 'A-F', 'A-G', 'A-H', 'A-I', 'A-J',
        'B-C', 'B-D', 'B-E', 'B-F', 'B-G', 'B-H', 'B-I', 'B-J',
        'C-D', 'C-E', 'C-F', 'C-G', 'C-H', 'C-I', 'C-J',
        'D-E', 'D-F', 'D-G', 'D-H', 'D-I', 'D-J',
        'E-F', 'E-G', 'E-H', 'E-I', 'E-J',
        'F-G', 'F-H', 'F-I', 'F-J',
        'G-H', 'G-I', 'G-J',
        'H-I', 'H-J',
        'I-J'
    ],
    'Ancho de Banda (Mbps)': [
        200, 250, 220, 280, 160, 240, 230, 220, 210,
        170, 210, 190, 160, 190, 240, 220, 220,
        180, 200, 190, 170, 200, 180, 220,
        220, 180, 200, 190, 210, 230,
        230, 210, 270, 280, 270,
        220, 200, 190, 280,
        210, 290, 180,
        200, 290, 210
    ]
}

# Crear el dataframe
df = pd.DataFrame(data)

# Ordenar las conexiones por Ancho de Banda (Mbps) en orden descendente
df_sorted = df.sort_values(by='Ancho de Banda (Mbps)', ascending=False)

# Crear un grafo vacío
G = nx.Graph()

# Función para verificar si la adición de una conexión crearía un ciclo
def creates_cycle(graph, node1, node2):
    # Solo verificar si ambos nodos ya están en el grafo
    if graph.has_node(node1) and graph.has_node(node2):
        return nx.has_path(graph, node1, node2)
    return False

# Iniciar el árbol de expansión máximo con la conexión de mayor ancho de banda
initial_connection = df_sorted.iloc[0]
ciudad1, ciudad2 = initial_connection['Ciudades Conectadas'].split('-')
G.add_edge(ciudad1, ciudad2, weight=initial_connection['Ancho de Banda (Mbps)'])

# Conjunto para llevar un registro de los nodos ya conectados
connected_nodes = {ciudad1, ciudad2}

# Recorrer las conexiones restantes y agregarlas al grafo si no forman un ciclo
for _, row in df_sorted.iloc[1:].iterrows():
    ciudad1, ciudad2 = row['Ciudades Conectadas'].split('-')

    # Verificar que al agregar la nueva conexión no se forme un ciclo
    if (ciudad1 in connected_nodes or ciudad2 in connected_nodes) and not creates_cycle(G, ciudad1, ciudad2):
        G.add_edge(ciudad1, ciudad2, weight=row['Ancho de Banda (Mbps)'])
        connected_nodes.update([ciudad1, ciudad2])

    # Detenerse cuando se hayan conectado todos los nodos
    if len(connected_nodes) == 10:  # Hay 10 ciudades: A, B, C, D, E, F, G, H, I, J
        break

# Calcular el valor total de Mbps
total_mbps = sum(edge[2]['weight'] for edge in G.edges(data=True))

# Mostrar las conexiones seleccionadas y el valor total de Mbps
print("Conexiones en el Árbol de Expansión Máximo:")
for edge in G.edges(data=True):
    print(f"{edge[0]}-{edge[1]}: {edge[2]['weight']} Mbps")

print(f"\nValor total de Mbps: {total_mbps} Mbps")



Conexiones en el Árbol de Expansión Máximo:
G-I: 290 Mbps
G-A: 240 Mbps
I-E: 280 Mbps
E-H: 270 Mbps
E-J: 270 Mbps
E-F: 230 Mbps
H-B: 240 Mbps
J-D: 230 Mbps
J-C: 220 Mbps

Valor total de Mbps: 2270 Mbps


Con este resultado ahora se realizara la busqueda local de la misma forma que en la solucion anterior.

Prompt:
ahora con esto quiero intentar encontrar una mejor solucion, asi que elimina de 2 en 2 las conexiones que se tienen actualmente y intenta todas las opciones posibles que se tengan. Recuerda que en la respuesta final solo pueden llegar a cambiar 2 partes de la solucion que ya se tiene

In [10]:

from itertools import combinations



# Calcular el valor total de Mbps
def calculate_total_mbps(graph):
    return sum(edge[2]['weight'] for edge in graph.edges(data=True))

total_mbps_original = calculate_total_mbps(G)

# Guardar la solución original
original_edges = list(G.edges(data=True))

# Función para generar una nueva solución eliminando 2 conexiones y probando otras
def try_alternatives(G, df, original_edges):
    best_graph = G.copy()
    best_mbps = calculate_total_mbps(G)

    # Obtener todas las combinaciones de 2 conexiones para eliminar
    for edges_to_remove in combinations(original_edges, 2):
        temp_graph = G.copy()

        # Eliminar las 2 conexiones seleccionadas
        temp_graph.remove_edge(edges_to_remove[0][0], edges_to_remove[0][1])
        temp_graph.remove_edge(edges_to_remove[1][0], edges_to_remove[1][1])

        # Buscar todas las posibles conexiones restantes en df_sorted
        remaining_edges = df[~df['Ciudades Conectadas'].isin([f"{edges_to_remove[0][0]}-{edges_to_remove[0][1]}", f"{edges_to_remove[1][0]}-{edges_to_remove[1][1]}"])]

        # Intentar agregar nuevas conexiones sin formar ciclos
        for new_edges in combinations(remaining_edges.itertuples(), 2):
            ciudad1, ciudad2 = new_edges[0][2].split('-')  # Acceso a 'Ciudades Conectadas' por índice
            ciudad3, ciudad4 = new_edges[1][2].split('-')

            if not creates_cycle(temp_graph, ciudad1, ciudad2):
                temp_graph.add_edge(ciudad1, ciudad2, weight=new_edges[0][3])  # 'Ancho de Banda (Mbps)' por índice

            if not creates_cycle(temp_graph, ciudad3, ciudad4):
                temp_graph.add_edge(ciudad3, ciudad4, weight=new_edges[1][3])

            # Verificar si esta nueva solución es mejor
            new_mbps = calculate_total_mbps(temp_graph)
            if new_mbps > best_mbps:
                best_graph = temp_graph.copy()
                best_mbps = new_mbps

    return best_graph, best_mbps

# Ejecutar el proceso de mejora
best_graph, best_total_mbps = try_alternatives(G, df_sorted, original_edges)

# Mostrar las conexiones seleccionadas y el valor total de Mbps
print("Conexiones en la mejor solución encontrada:")
for edge in best_graph.edges(data=True):
    print(f"{edge[0]}-{edge[1]}: {edge[2]['weight']} Mbps")

print(f"\nValor total de Mbps de la mejor solución: {best_total_mbps} Mbps")
print(f"Valor total de Mbps de la solución original: {total_mbps_original} Mbps")


Conexiones en la mejor solución encontrada:
G-I: 290 Mbps
I-E: 280 Mbps
E-H: 270 Mbps
E-J: 270 Mbps
E-A: 280 Mbps
H-B: 240 Mbps
J-D: 230 Mbps
J-C: 220 Mbps
J-F: 280 Mbps

Valor total de Mbps de la mejor solución: 2360 Mbps
Valor total de Mbps de la solución original: 2270 Mbps


## Comparacion entre soluciones:

La solucion 1 genero la respuesta mas optima posible para este problema, consiguiendo un valor de mbps de 2410, mientras que nuestra segunda solucion consiguio un valor original de 2270mbps, mostrando una clara falla en calidad de resultados en comparacion a nuestra primera solucion.

En general consideraria que la primera solucion ademas de conseguir mejores resultados es la opcion mas eficiente, esto ya que el numero de veces que se tienen que leer los datos es mucho menor que en la segunda solucion, teniendo que leer unicamente hasta el siguiente valor mas alto no utilizado. En el caso de mi segunda solucion el numero de veces que se tienen que hacer ciclos no afecta mucho en el numero de datos evaluados, pero conforme se continua con el codigo el procesamiento de respuestas se vuelve mas pesado, por lo que en caso de ser un problema con una mayor cantidad de nodos es posible que se tengan una gran cantidad de problemas al tener una expansion de soluciones evaluadas en forma cuadratica.

Despues de haber realizado la busqueda local nuestra segunda respuesta logro acercarse mucho mas a los valores que se consiguieron en nuestra solucion inicial, consiguiendo llegar hasta un valor de 2360 Mbps, teniendo una diferencia unicamente de 50 Mbps con la solucion mas eficiente posible.

## Posibilidad de encontrar mejores soluciones
En el caso de nuestra primera solucion es imposible que se puedan llegar a encontrar mejores soluciones ya que desde el inicio se llego a la respuesta mas optima.

En nuestra segunda solucion existe un potencial de mejora el cual podria llegar a ser conseguible usando la misma metodologia de mejora que se aplico en nuestra busqueda local, pero esta puede llegar a tener sus limites. Para tener una mejora en este procedimiento seria necesario el agregar un tercer nodo el cual se elimine para evaluar mas posibles respuestas que se podrian llegar a tener, ya que el cambio de 2 nodos podria llegar a mostrarse como insuficiente.
